# Run a blueprint in C-star

This example demonstrates how to launch a regional ocean model simulation including biogeochemical processes using ROMS-MARBL, from a roms-marbl blueprint.\


## Prerequisities

In order to run this notebook, you need to have:

- [Installed](installation.rst) and [configured](configuration.rst) `C-Star` on a laptop or HPC
- Activated the environment in your terminal with `conda activate cstar_env`
- Created the ipykernel using:

  `python -m ipykernel install --user --name cstar_env --display-name cstar_env`
- Launched this notebook with `jupyter tutorial_bp.ipynb` and selected the `cstar_env` in the kernel dropdown

To run a the ROMS-MARBL application, the following files first need to be created:
* Input data files (`.yaml` or `.nc`)
* Compile-time files (`.opt`) 
* Runtime files (`.in`, `marbl_*`) files

For this example, we provide a blueprint along with all files needed to run the simulation.\
The blueprint in this example can be downloaded [here](https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example/raw/refs/heads/main/wio-toy-domain/wio_toy_blueprint.yaml), or simply copy the yaml contents below into a file named `wio_toy_blueprint.yaml` in your working directory.

To create your own simulation, consider using this blueprint as a template and creating your own files using [Roms-Tools](https://roms-tools.readthedocs.io/en/latest/index.html) for input data files, or [C-SON Forge](https://cworthy-ocean.github.io/cson-forge/installation/) for all necessary files (input, compile-time, run-time).

## Inspect and run the simulation

<div class="alert alert-info">
    
Note

`C-Star` is a command line utility. Here we have it in a Jupyter Notebook to easily step through commands and see results as they run. These commands can equally be run in the command line by removing the '!' from in front on each command. 
</div>

### Define the blueprint

We can print the content of the blueprint with `cat wio_toy_blueprint.yaml`, and see that each of the attributes described in the [blueprints page](blueprints.rst) are defined.

To reference the file architecture and what this blueprint points to, see this [repository](https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example/tree/main/wio-toy-domain).\
For a complete list of available parameters, see the [Blueprint API page](api-blueprint).

**Print out the blueprint contents:**

In [5]:
!cat wio_toy_blueprint.yaml

name: wio_toy
description: wio toy domain for laptop
application: roms_marbl
state: draft
valid_start_date: 2012-01-01
valid_end_date: 2012-01-02

code:
  roms:
    location: https://github.com/CWorthy-ocean/ucla-roms.git
    branch: a338198af93a7b4cfa8f320a23f0f5623bc18152
  marbl:
    location: https://github.com/marbl-ecosys/MARBL.git
    branch: marbl0.45.0

  run_time:
    location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
    branch: main
    filter:
      directory: wio-toy-domain/runtime_code
      files:
        - cson_roms-marbl_v0.1_wio-toy_10procs.in
        - marbl_in
        - marbl_tracer_output_list
        - marbl_diagnostic_output_list
  compile_time:
    location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
    branch: main
    filter:
      directory: wio-toy-domain/compile_time_code
      files:
        - bgc.opt
        - bulk_frc.opt
        - cdr_frc.opt
        - cppdefs.opt
        - diagnostics.opt
  

### Models
The specific versions of the models (ROMS and MARBL) used for the simulation are definded under the `code` block, where `location` can also point to specific forks of the repository, and `branch` or `commit` can be used to specify a unique instance of the code.

### Runtime code
For the files needed at the run-time of the model, the blueprint can either point to a `git` repository online, as shown above under `run_time`, or to a local directory (see example [here](blueprints.rst#romsmarblblueprint-example)). While the runtime file `*.in` is classically used to launch ROMS when running it stand-alone, `C-star` only extracts the parameter values (`bottom_drag`, `S-coord`, etc.) and relies on the blueprint for time-stepping and file locations of input data files.

### Compile-time code
Like the run-time files, the blueprint can point to an online repo or a local directory for the files needed to compile the model. All specified under `compile_time`.

### Forcing files
The location of each of the input data files are specified in their respective sections - `grid`, `intitial_conditions`, `tidal`, `surface`, etc.\
The `location` can point to either a YAML or netCDF file. If a netCDF file is specified, `C-Star` can partition the file for you, by specifying `partitioned: false` (see example usage of `partition` [here](blueprints.rst#romsmarblblueprint-example), where the files are already partitioned).

The files will be paritioned according to the parameters defined under `partitioning` (i.e. `n_procs_x`, `n_procs_y`).\
However, if the files are already partitioned, these parameters must match how many horizontal (`n_procs_x`) and vertical (`n_procs_y`) partitions they were divided into. 

### Model and runtime parameters

The time step to be used in the model is specified under `model_params`. This example uses a 900 second time step.\
The start and end date of the simulation are defined under `runtime_params` along with the location in which to store the output files from both the simulation run and the `C-Star` orchestration. The start and end dates must be within the range of the `valid_start_date` and `valid_end_date`. 

## Check the blueprint for syntax

Once the blueprint is complete, we can use `C-Star` to proofread it. The `cstar blueprint` command has a functionality `check` to confirm that the syntax of the blueprint YAML is correct and all necessary parameters are included.\
This does not check for things such as whether the input files exist, etc.

In [6]:
!cstar blueprint check wio_toy_blueprint.yaml

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
The blueprint is valid


If the output directory remains from a previous run, it may need to be remove before running the simulation.

In [7]:
!rm -rf output

## Run the model from the blueprint

In [7]:
!cstar blueprint run wio_toy_blueprint.yaml

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Executing blueprint in a worker service
2026-02-06 17:21:09,958 [DEBUG] - worker.py:459 - Job config: JobConfig(account_id='None', walltime='0:10:00', job_name='None', priority='regular')
2026-02-06 17:21:09,958 [DEBUG] - worker.py:460 - Simulation runner service config: as_service=True loop_delay=5.0 health_check_frequency=None log_level=10 health_check_log_threshold=10 name='SimulationRunner' max_health_check_latency=1.0
2026-02-06 17:21:09,958 [DEBUG] - worker.py:461 - Simulation request: BlueprintRequest(blueprint_uri='wio_toy_blueprint.yaml', stages=[<SimulationStages.SETUP: 'setup'>, <SimulationStages.BUILD: 'build'>, <SimulationStages.PRE_RUN: 'pre_run'>, <SimulationStages.RUN: 'run'>, <SimulationStages.POST_RUN: 'post_run'>])
2026-02-06 17:21:09,958 [DEBUG] - worker.py:462 - os.environ: environ({'JPY_SESSION_NAME': '/Users/sam.maticka/workdir/cstar_blueprint_roms_marbl_example/wio-t

## Checking results

All of the output files from the model runs can be found in subdirectories under the `output_dir` specified in the blueprint. 

The contents in each of the subdirecties are:
- **codebase**:  The model source code pulled from the location specified in the blueprint, and compiled by C-Star.
- **compile_time_code**:  The compile-time files specified in the blueprint and the ROMS executable resulting from compiling the code under `codebase`.
- **input_datasets**:  The partitioned input data files used in the model. These are either links to user-paritioned files specified in the blueprint, or partitioned versions of unpartitioned files specified in the blueprint.
- **runtime_code**:  The runtime files provided by the user (i.e. `marbl_*`, `.in` files), and the `C-Star`-modified `.in` file.
- **logs**:  The `.out` log files. This comes from the output of the model. Errors in this file will come directly from `ROMS` or `MARBL`. 
- **joined_output**:  Joined output result files. These files can be inspected using `Roms-Tools`.
- **output**:  The raw unpartitioned output files from the models.
- **work**: